In [5]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import random

# Source and destination directories
source_dir = r'D:\project\image_forging\real'
destination_dir = r'D:\project\image_forging\fake'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Load all images from the source directory
image_files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

def copy_move_forgery(image):
    """ Copy and paste a random patch within the same image """
    width, height = image.size
    x1, y1 = random.randint(0, width // 2), random.randint(0, height // 2)
    patch_width, patch_height = random.randint(20, 50), random.randint(20, 50)
    
    patch = image.crop((x1, y1, x1 + patch_width, y1 + patch_height))
    
    x2, y2 = random.randint(0, width - patch_width), random.randint(0, height - patch_height)
    image.paste(patch, (x2, y2))
    return image

def splicing(image, original_image_filename):
    """ Combine two images into one composite image """
    other_image_file = random.choice([f for f in image_files if f != original_image_filename])
    other_image_path = os.path.join(source_dir, other_image_file)
    other_image = Image.open(other_image_path).resize(image.size)
    mask = Image.new("L", image.size, 128)
    return Image.composite(image, other_image, mask)

def image_retouching(image):
    """ Adjust color, brightness, contrast or texture """
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5))
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(random.uniform(0.5, 1.5))

def resizing_and_warping(image):
    """ Resize or warp objects in the image """
    width, height = image.size
    scale = random.uniform(0.5, 1.5)
    resized_image = image.resize((int(width * scale), int(height * scale)))
    return resized_image.resize((width, height))

def inpainting(image):
    """ Remove random object and fill the space """
    image_array = np.array(image)
    mask = np.zeros(image_array.shape[:2], np.uint8)
    x, y = random.randint(20, image_array.shape[1] - 20), random.randint(20, image_array.shape[0] - 20)
    mask[y:y+50, x:x+50] = 255
    
    inpainted_image = cv2.inpaint(image_array, mask, 3, cv2.INPAINT_TELEA)
    return Image.fromarray(inpainted_image)

def simple_face_swap(image):
    """ Simple face swap (only works if faces are detected) """
    image_array = np.array(image)
    gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) >= 2:
        # Get coordinates for two faces
        x1, y1, w1, h1 = faces[0]
        x2, y2, w2, h2 = faces[1]
        
        # Extract face regions
        face1 = image.crop((x1, y1, x1 + w1, y1 + h1))
        face2 = image.crop((x2, y2, x2 + w2, y2 + h2))
        
        # Resize face1 to match the size of face2 and vice versa
        face1_resized = face1.resize((w2, h2))
        face2_resized = face2.resize((w1, h1))
        
        # Paste resized faces back into the image
        image.paste(face1_resized, (x2, y2))
        image.paste(face2_resized, (x1, y1))
    
    return image

def contrast_manipulation(image):
    """ Change contrast to highlight or obscure details """
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(0.5, 2.0)  # Randomly change contrast
    return enhancer.enhance(factor)

def watermark_removal(image):
    """ Add and remove a watermark """
    watermark = "Sample"
    image_array = np.array(image)
    cv2.putText(image_array, watermark, (20, image_array.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return Image.fromarray(image_array)

def image_morphing(image, original_image_filename):
    """ Blend the image with another randomly chosen image """
    other_image_file = random.choice([f for f in image_files if f != original_image_filename])
    other_image_path = os.path.join(source_dir, other_image_file)
    other_image = Image.open(other_image_path).resize(image.size)
    alpha = random.uniform(0.3, 0.7)
    blended_image = Image.blend(image, other_image, alpha)
    return blended_image

# List of forging techniques
forging_techniques = [
    copy_move_forgery, splicing, image_retouching, 
    inpainting, simple_face_swap, image_morphing
]

# Counter for total forged images
total_forged_images = 0

# Loop through each image file and apply forging techniques
for image_file in image_files:
    original_image_path = os.path.join(source_dir, image_file)
    image = Image.open(original_image_path)

    # Generate forged images using different techniques
    for i in range(4):  # Adjust this number to generate more forged images
        forged_image = image.copy()
        technique = random.choice(forging_techniques)
        
        if technique in [splicing, image_morphing]:
            forged_image = technique(forged_image, image_file)  # Pass filename for these techniques
        else:
            forged_image = technique(forged_image)
        
        # Save the forged image
        forged_image_filename = f"forged_{total_forged_images + 1}_{image_file}"
        forged_image.save(os.path.join(destination_dir, forged_image_filename))
        
        total_forged_images += 1
        
        # Stop if we've created at least 250 forged images
        if total_forged_images >= 250:
            break
    
    if total_forged_images >= 250:
        break

print(f"Total forged images created: {total_forged_images}")


Total forged images created: 224


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random

# Source and destination directories
source_dir = r'D:\project\image_forging\real'
destination_dir = r'D:\project\image_forging\fake'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Load all images from the source directory
image_files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

def rotate_image(image):
    """ Rotate the image randomly between -45 to 45 degrees """
    angle = random.uniform(-45, 45)
    return image.rotate(angle, expand=True)

def flip_image(image):
    """ Flip the image either horizontally or vertically """
    flip_type = random.choice(["horizontal", "vertical"])
    if flip_type == "horizontal":
        return image.transpose(Image.FLIP_LEFT_RIGHT)
    else:
        return image.transpose(Image.FLIP_TOP_BOTTOM)

def add_noise(image):
    """ Add random Gaussian noise to the image """
    image_array = np.array(image)
    mean = 0
    std_dev = 25
    noise = np.random.normal(mean, std_dev, image_array.shape).astype(np.uint8)
    noisy_image = cv2.add(image_array, noise)
    return Image.fromarray(noisy_image)

def change_brightness(image):
    """ Adjust the brightness of the image """
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(0.5, 1.5)  # Brightness factor between 0.5 and 1.5
    return enhancer.enhance(factor)

def apply_blur(image):
    """ Apply Gaussian blur to the image """
    return image.filter(ImageFilter.GaussianBlur(radius=random.uniform(1, 3)))

def crop_image(image):
    """ Randomly crop the image """
    width, height = image.size
    crop_width = random.randint(int(width * 0.6), width)
    crop_height = random.randint(int(height * 0.6), height)
    left = random.randint(0, width - crop_width)
    top = random.randint(0, height - crop_height)
    return image.crop((left, top, left + crop_width, top + crop_height))

# List of forging techniques
forging_techniques = [rotate_image, flip_image, add_noise, change_brightness, apply_blur, crop_image]

# Counter for total forged images
total_forged_images = 0

# Loop through each image file and apply forging techniques
for image_file in image_files:
    original_image_path = os.path.join(source_dir, image_file)
    image = Image.open(original_image_path)

    # Generate forged images using different techniques
    for i in range(3):  # Adjust this number to generate more forged images
        forged_image = image.copy()
        technique = random.choice(forging_techniques)
        forged_image = technique(forged_image)
        
        # Save the forged image
        forged_image_filename = f"forged_{total_forged_images + 1}_{image_file}"
        forged_image.save(os.path.join(destination_dir, forged_image_filename))
        
        total_forged_images += 1
        
        # Stop if we've created at least 150 forged images
        if total_forged_images >= 150:
            break
    
    if total_forged_images >= 150:
        break

print(f"Total forged images created: {total_forged_images}")
